<a href="https://colab.research.google.com/github/jonathannocek/pga-data-analysis/blob/master/pga_data_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS377: Data Science Final Project: Data Scraping**


This data scraping function is modified from [here](https://github.com/daronprater/PGA-Tour-Data-Science-Project/blob/master/PGAtour.com%20Web%20Scraper.ipynb).

## **Imports**

In [0]:
import requests 
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup # Used for pulling data out of HTML files
import seaborn as sns

#### **Get Column Headers**

This function will get the column names to use for the dataframe. Using BeautifulSoup, it will pull all headers with the rounds class and then, all headers with the col-stat class. Finally, it will save all of this into a headers list and return this list. 

In [0]:
def get_headers(soup):
    headers = [] #Initialize empty list for column names
    
    #Get rounds header
    rounds = soup.find_all(class_="rounds hidden-small hidden-medium")[0].get_text()
    headers.append(rounds)
    
    #Get other headers
    stat_headers = soup.find_all(class_="col-stat hidden-small hidden-medium")
    for header in stat_headers:
        headers.append(header.get_text())
    
    return headers

#### **Get Players Names**

This function takes the BeautifulSoup created and uses it to gather player's names

In [0]:
def get_players(soup):    
    player_list = [] #Initialize empty list for players
    
    #Get player as html tags
    players = soup.select('td a')[1:] #Use 1 beacuse first line of all tables is not useful.
    
    #Loop through list
    for player in players:
        player_list.append(player.get_text())
    
    return player_list

### **Get Stats**

Will find all of the statistics specified and compile them into a list. 


In [0]:
def get_stats(soup, categories):
    '''This function takes the soup created before and the number of categories 
    needed to generate this'''
    
    #Finds all tags with class specified and puts into a list
    stats = soup.find_all(class_="hidden-small hidden-medium")
    print(stats)
    
    #Initialize stats list
    stat_list = []
    
    #Loop through 
    for i in range(0, len(stats)-categories+1, categories):
        temp_list = []
        for j in range(categories):
            temp_list.append(stats[i + j].get_text())
        stat_list.append(temp_list)
            
    return stat_list

### **Create Stats Dictionary**

This function take two lists, players and stats, and combines them into a dictionary with the player as the key.

In [0]:
def stats_dict(players, stats):
        '''This function takes two lists, players and stats, 
        and creates a dictionary with the player being the key 
        and the stats as the vales (as a list)'''
    
        #initialize player dictionary
        player_dict = {}

        #Loop through player list
        for i, player in enumerate(players):
            player_dict[player] = stats[i]
    
        return player_dict

### **Creating the Dataframe**

make_dataframe will take the url where the desired data is located and the number of the category. 

The function will get headers, get players, get stats, and finally combine them into a dictionary. From there, it will use pandas to create a dataframe with the dictionary and the headers as column names. 

In [0]:
def make_dataframe(url, categories):
        
    ##Create soup object from url.
    response = requests.get(url)
    text = response.text
    soup = BeautifulSoup(text, 'lxml')
    
    #1. Get Headers
    headers = get_headers(soup)
    
    #2. Get Players
    players = get_players(soup)
    
    #3. Get Stats
    stats = get_stats(soup, categories)
    
    #4. Make stats dictionary.
    stats_dictionary = stats_dict(players, stats)
    
    #Make dataframe
    frame = pd.DataFrame(stats_dictionary, index = headers).T
    
    #Reset index
    frame = frame.reset_index()
    
    #For each Dataframe, change index column to 'NAME'
    frame = frame.rename(index = str, columns = {'index': 'NAME'})
    return frame

Use the following below to specify which years of statistics are being scraped. 

In [0]:
years = [str(i) for i in range(2010, 2020)]

### **Putting it all together**

Here, the for loop iterates through the years 2010-2019 to find the following statisitcs:
1. FedEx Cup points
2. Top 10s
3. Wins
4. Scoring Average
5. Driving Distance
6. Driving Driving Accuracy
7. Green in Regulations
8. Strokes Gained - Tee to Green
9. Strokes Gained - Off the Tee
10. Strokes Gained - Approach the Green
11. Strokes Gained - Around the Green
12. Strokes Gained - Putting
13. Strokes Gained - Total
14. Par 3 Scoring
15. Par 4 Scoring
16. Par 5 Scoring 

In [0]:
for year in years:
    print(year)
    #Fedex cup points
    fcp = make_dataframe("https://www.pgatour.com/stats/stat.02671.y{}.html".format(year), 6)[['NAME', 'POINTS']]
    #Top 10's and wins
    top10 = make_dataframe("https://www.pgatour.com/stats/stat.138.y{}.html".format(year), 4)[['NAME', '1ST']]

    # Scoring statistics, keep rounds from this page as it most accurately reflects total rounds player completed in season.
    scoring = make_dataframe("https://www.pgatour.com/stats/stat.120.y{}.html".format(year), 5)[['NAME', 'ROUNDS', 'AVG']]
    scoring = scoring.rename(columns={'AVG':'SCORING'})

    # Driving Distance
    drivedistance = make_dataframe("https://www.pgatour.com/stats/stat.101.y{}.html".format(year), 4)[['NAME', 'AVG.']]
    #Rename Columns
    drivedistance = drivedistance.rename(columns = {'AVG.':'DRIVE_DISTANCE'})

    # Driving Accuracy
    driveacc = make_dataframe("https://www.pgatour.com/stats/stat.102.y{}.html".format(year), 4)[['NAME', '%']]
    # Change column name from % to FWY %
    driveacc = driveacc.rename(columns = {'%': "FWY_%"})

    # Greens in Regulation.
    gir = make_dataframe("https://www.pgatour.com/stats/stat.103.y{}.html".format(year), 5)[['NAME', '%']]
    # Change column name from % to GIR %
    gir = gir.rename(columns = {'%': "GIR_%"})

    # SG-Tee tp Green
    sg_teetogreen = make_dataframe("https://www.pgatour.com/stats/stat.02674.y{}.html".format(year), 6)[['NAME', 'AVERAGE']]
    #Change name of average column
    sg_teetogreen = sg_teetogreen.rename(columns = {'AVERAGE' : 'SG_TTG'})

    # SG-Off the Tee
    sg_ott = make_dataframe("https://www.pgatour.com/stats/stat.02567.y{}.html".format(year), 4)[['NAME', 'AVERAGE']]
    #Rename Columns
    sg_ott = sg_ott.rename(columns = {'AVERAGE':'SG_OTT'})
    
    # SG-Approach
    sg_approach = make_dataframe("https://www.pgatour.com/stats/stat.02568.y{}.html".format(year), 4)[['NAME', 'AVERAGE']]
    #Rename Columns
    sg_approach = sg_approach.rename(columns = {'AVERAGE':'SG_APP'})

    # SG-Around the Green
    sg_around = make_dataframe("https://www.pgatour.com/stats/stat.02569.y{}.html".format(year), 4)[['NAME', 'AVERAGE']]
    #Rename Columns
    sg_around = sg_around.rename(columns = {'AVERAGE':'SG_ATG'})

    # SG-Putting
    sg_putting = make_dataframe("https://www.pgatour.com/stats/stat.02564.y{}.html".format(year), 4)[['NAME', 'AVERAGE']]
    #Change name of average column
    sg_putting = sg_putting.rename(columns = {'AVERAGE': 'SG_P'})
   
    # SG-Total
    sg_total = make_dataframe("https://www.pgatour.com/stats/stat.02675.y{}.html".format(year), 6)[['NAME', 'AVERAGE']]
    sg_total = sg_total.rename(columns = {'AVERAGE':'SG_T'})

    # Par 3 Scoring
    par3_scoring = make_dataframe("https://www.pgatour.com/stats/stat.142.y{}.html".format(year), 4)[['NAME', 'AVG']]
    par3_scoring = par3_scoring.rename(columns = {'AVG':'PAR3'})
    
    # Par 4 Scoring
    par4_scoring = make_dataframe("https://www.pgatour.com/stats/stat.143.y{}.html".format(year), 4)[['NAME', 'AVG']]
    par4_scoring = par4_scoring.rename(columns = {'AVG':'PAR4'})

    # Par 5 Scoring
    par5_scoring = make_dataframe("https://www.pgatour.com/stats/stat.144.y{}.html".format(year), 4)[['NAME', 'AVG']]
    par5_scoring = par5_scoring.rename(columns = {'AVG':'PAR5'})

    #Get Dataframes into list.
    data_frames = [drivedistance, driveacc, gir, sg_putting, sg_around, sg_approach, sg_ott, sg_teetogreen, sg_total, par3_scoring, par4_scoring, par5_scoring]
    
    #Merge all Dataframes together
    df_one = pd.DataFrame()
    df_one = scoring
    for df in data_frames:
        df_one = pd.merge(df_one, df, on='NAME')
        
    

    #merge fex ex cup points
    df_one = pd.merge(df_one, fcp, how='outer', on='NAME')
    #Merge top 10's
    df_one = pd.merge(df_one, top10, how='outer', on='NAME')
    
    #Only get people who's scoring average isn't null.
    df_one = df_one.loc[df_one['SCORING'].isnull() == False]  
    
    #Add year column
    df_one['Year'] = year
    
    #Concat dataframe to overall dataframe
    if year == '2010':
        df_total = pd.DataFrame()
        df_total = pd.concat([df_total, df_one], axis=0)
    else:
        df_total = pd.concat([df_total, df_one], axis=0)

2010
[<span class="hidden-small hidden-medium">THIS WEEK</span>, <td class="hidden-small hidden-medium">21</td>, <td class="hidden-small hidden-medium">3</td>, <td class="hidden-small hidden-medium">7</td>, <td class="hidden-small hidden-medium"></td>, <td class="hidden-small hidden-medium"></td>, <td class="hidden-small hidden-medium">25</td>, <td class="hidden-small hidden-medium">1</td>, <td class="hidden-small hidden-medium">11</td>, <td class="hidden-small hidden-medium">252</td>, <td class="hidden-small hidden-medium"></td>, <td class="hidden-small hidden-medium">20</td>, <td class="hidden-small hidden-medium">0</td>, <td class="hidden-small hidden-medium">7</td>, <td class="hidden-small hidden-medium">280</td>, <td class="hidden-small hidden-medium"></td>, <td class="hidden-small hidden-medium">23</td>, <td class="hidden-small hidden-medium">1</td>, <td class="hidden-small hidden-medium">5</td>, <td class="hidden-small hidden-medium">480</td>, <td class="hidden-small hidden-medi

IndexError: ignored

In [0]:
df_total.head(15)

,NAME,ROUNDS,SCORING,DRIVE_DISTANCE,FWY_%,GIR_%,SG_P,SG_ATG,SG_APP,SG_OTT,SG_TTG,SG_T,PAR3,PAR4,PAR5,POINTS,TOP 10,1ST,Year
0,Matt Kuchar,97,69.606,286.9,67.89,69.36,.648,.334,.336,.158,.827,1.461,3.02,3.96,4.56,"2,728",11,1,2010
1,Steve Stricker,73,69.660,282.9,68.50,68.29,.437,.419,.773,.191,1.383,1.818,2.99,3.99,4.58,"2,028",9,2,2010
2,Retief Goosen,75,69.718,291.4,64.79,65.96,.679,.395,.185,.337,.917,1.598,3.04,3.99,4.60,"1,360",10,,2010
3,Paul Casey,64,69.720,294.2,61.31,68.68,.812,-.111,.483,.215,.587,1.411,3.03,3.98,4.64,"2,250",7,,2010
4,Jim Furyk,76,69.828,276.0,71.01,67.12,.402,.367,.641,.150,1.159,1.564,3.02,3.99,4.70,"2,980",7,3,2010
5,Ernie Els,72,69.843,288.4,60.16,67.86,.330,.043,.735,.215,.992,1.322,3.13,3.98,4.63,"1,438",7,2,2010
6,Luke Donald,71,69.850,277.0,62.36,65.28,.870,.464,.661,-.506,.619,1.493,3.00,4.00,4.71,"2,700",7,,2010
7,Justin Rose,78,69.885,287.8,65.17,66.31,.243,.447,.168,.338,.952,1.195,3.03,4.00,4.63,718,4,2,2010
8,Bo Van Pelt,104,69.955,292.0,65.23,69.23,.098,.107,.260,.724,1.091,1.192,3.06,3.99,4.61,445,8,,2010
9,Phil Mickelson,76,69.966,299.1,52.66,65.13,-.147,.228,.738,.185,1.151,1.001,3.05,4.01,4.58,843,6,1,2010


In [0]:
df_total.describe()

,NAME,ROUNDS,SCORING,DRIVE_DISTANCE,FWY_%,GIR_%,SG_P,SG_ATG,SG_APP,SG_OTT,SG_TTG,SG_T,PAR3,PAR4,PAR5,POINTS,TOP 10,1ST,Year
count,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1861,1540,1540,1866
unique,465,73,1299,401,1195,835,993,774,1036,1005,1261,1278,30,29,46,1037,14,6,10
top,Webb Simpson,81,70.966,288.1,60.10,66.67,.000,-.111,.054,.079,-.003,.030,3.06,4.04,4.67,142,1,,2018
freq,10,57,6,16,6,22,7,9,7,6,7,6,182,210,135,7,390,1194,193


In [0]:
from google.colab import files

df_total.to_csv('2010_2019_PGA_Stats.csv')
files.download('2010_2019_PGA_Stats.csv')